# data set 
Foi pego no uci, convertido para csv com auxilio de script


In [0]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import KFold
from statistics import mean 
from collections import Counter
from sklearn.metrics import confusion_matrix


In [0]:
headers = list(pd.read_csv("EGG_EyeStateHeaders.csv") )

In [4]:
data = pd.read_csv("EEG_EyeState.csv", names=headers, index_col=False, skiprows = [0]) 
# Preview the first 5 lines of the loaded data 
data.head()

,AF3,F7,F3,FC5,T7,P7,O1,O2,P8,T8,FC6,F4,F8,AF4,eyeDetection
0,4329.23,4009.23,4289.23,4148.21,4350.26,4586.15,4096.92,4641.03,4222.05,4238.46,4211.28,4280.51,4635.90,4393.85,0
1,4324.62,4004.62,4293.85,4148.72,4342.05,4586.67,4097.44,4638.97,4210.77,4226.67,4207.69,4279.49,4632.82,4384.10,0
2,4327.69,4006.67,4295.38,4156.41,4336.92,4583.59,4096.92,4630.26,4207.69,4222.05,4206.67,4282.05,4628.72,4389.23,0
3,4328.72,4011.79,4296.41,4155.90,4343.59,4582.56,4097.44,4630.77,4217.44,4235.38,4210.77,4287.69,4632.31,4396.41,0
4,4326.15,4011.79,4292.31,4151.28,4347.69,4586.67,4095.90,4627.69,4210.77,4244.10,4212.82,4288.21,4632.82,4398.46,0


In [0]:
X_train, X_test, y_train, y_test = train_test_split(data, data.eyeDetection, test_size=0.3)                                                    
X_train = X_train.drop("eyeDetection", axis=1)
X_test = X_test.drop("eyeDetection", axis=1)

In [0]:
# print(X_train)
# print(X_test)
# print(y_train)
# print(y_test)

Algoritmo diferente para a harmonica

In [0]:
from scipy.spatial import distance as dist_lib
def euclidean(instance1, instance2):
    return dist_lib.euclidean(instance1, instance2)

def canberra(instance1, instance2):
    return dist_lib.canberra(instance1, instance2)

def chebyshev(instance1, instance2):
    return dist_lib.chebyshev(instance1, instance2)

def correlation(instance1, instance2):
    return dist_lib.correlation(instance1, instance2)

def cosine(instance1, instance2):
    return dist_lib.cosine(instance1, instance2)

In [0]:
def get_neighbors(training_set, 
                  labels, 
                  test_instance, 
                  k, 
                  distance):
    """
    get_neighors calculates a list of the k nearest neighbors
    of an instance 'test_instance'.
    The list neighbors contains 3-tuples with  
    (index, dist, label),
    where:
    index    is the index from the training_set, 
    dist     is the distance between the test_instance 
             and the instance training_set[index]
    distance is a reference to a function used to calculate the 
             distances
    """
    distances = []
    for index, row in training_set.iterrows():
        dist = distance(test_instance, row)
        distances.append((row, dist, labels[index]))
    distances.sort(key=lambda x: x[1])
    neighbors = distances[:k]
    return neighbors


            
def vote_harmonic_weights(neighbors, all_results=True):
    class_counter = Counter()
    number_of_neighbors = len(neighbors)
    
    # para cada classe, somam-se os inversos de suas ordens
    for index in range(number_of_neighbors):
        class_counter[neighbors[index][2]] += 1/(index+1)
    labels, votes = zip(*class_counter.most_common())
    winner = class_counter.most_common(1)[0][0]
    votes4winner = class_counter.most_common(1)[0][1]
    if all_results:
        total = sum(class_counter.values(), 0.0)
        for key in class_counter:
             class_counter[key] /= total
        return winner, class_counter.most_common()
    else:
        return winner, votes4winner / sum(votes)
    
def knn_test_harmonic(trainset_data, trainset_labels, testset_data, knn_k, distance_function):
    print(testset_data.shape[0])
    count_row = 0
    df_prediction = pd.DataFrame(columns=['class'])
    for index, row in testset_data.iterrows():
        neighbors = get_neighbors(trainset_data, 
                                  trainset_labels, 
                                  row, 
                                  knn_k, 
                                  distance=distance_function)
        #print(vote_harmonic_weights(neighbors, all_results=True))
        predited_class = vote_harmonic_weights(neighbors, all_results=True)[0]
        df_prediction.loc[index] = predited_class
        count_row += 1
        if (count_row % 50 == 0):
            print(count_row, count_row/testset_data.shape[0])
    
    return df_prediction

In [0]:
weights = [#'uniform', 'distance', 
    'harmonic']

distances = ['canberra', 'chebyshev', 'correlation', 'euclidean','cosine' ] #'mahalanobis'
V = np.cov(X_train)

In [0]:
def cross_validation(knn_k, distance, weight_function):
    if (weight_function == 'harmonic'):
        kf = KFold(n_splits= 5, shuffle=True)
        accs = []
        for train_index, test_index in kf.split(X_train):
            fold_x_train, fold_x_test = X_train[X_train.index.isin(train_index)], X_train[X_train.index.isin(test_index)]
            fold_y_train, fold_y_test = y_train[y_train.index.isin(train_index)], y_train[y_train.index.isin(test_index)]
                
            print("Treino: " + str(X_train.shape[0]) + ", fold_treino: " + str(fold_x_train.shape[0]) + ", fold_test: "+ str(fold_x_test.shape[0]))
            
            #Train the model using the training sets
            if (distance == 'canberra'):
                fold_y_pred = knn_test_harmonic(fold_x_train, fold_y_train, fold_x_test, knn_k, canberra)
            elif (distance == 'chebyshev'):
                fold_y_pred = knn_test_harmonic(fold_x_train, fold_y_train, fold_x_test, knn_k, euclidean)
            elif (distance == 'correlation'):
                fold_y_pred = knn_test_harmonic(fold_x_train, fold_y_train, fold_x_test, knn_k, euclidean)
            elif (distance == 'euclidean'):
                fold_y_pred = knn_test_harmonic(fold_x_train, fold_y_train, fold_x_test, knn_k, euclidean)
            elif (distance == 'cosine'):
                fold_y_pred = knn_test_harmonic(fold_x_train, fold_y_train, fold_x_test, knn_k, euclidean)
                
            acc = metrics.accuracy_score(fold_y_test, fold_y_pred['class'].astype('int32'))
            # Model Accuracy, how often is the classifier correct?
            print("Accuracy:", acc)
            accs.append(acc)
            cm = confusion_matrix(fold_y_test, fold_y_pred['class'].astype('int32'))
            print(cm)
        
        return [knn_k, distance, weight_function, mean(accs)]
            
    else:
        #Create KNN Classifier
        if distance == 'mahalanobis':
            knn = KNeighborsClassifier(n_neighbors=knn_k, metric=distance, weights = weight_function, metric_params={'V': V})
        else:
            knn = KNeighborsClassifier(n_neighbors=knn_k, metric=distance, weights = weight_function)

        kf = KFold(n_splits= 5, shuffle=True)
        accs = []
        for train_index, test_index in kf.split(X_train):
            fold_x_train, fold_x_test = X_train[X_train.index.isin(train_index)], X_train[X_train.index.isin(test_index)]
            fold_y_train, fold_y_test = y_train[y_train.index.isin(train_index)], y_train[y_train.index.isin(test_index)]
            #Train the model using the training sets
            knn.fit(fold_x_train, fold_y_train)

            #Predict the response for test dataset
            fold_y_pred = knn.predict(fold_x_test)

            acc = metrics.accuracy_score(fold_y_test, fold_y_pred)
            # Model Accuracy, how often is the classifier correct?
            print("Accuracy:", acc)
            accs.append(acc)
            cm = confusion_matrix(y_test, y_pred)
    
        return [knn_k, distance, weight_function, mean(accs)]

In [0]:
def test(knn_k, distance, weight_function):
    #Create KNN Classifier
    if distance == 'mahalanobis':
        knn = KNeighborsClassifier(n_neighbors=knn_k, metric=distance, weights = weight_function, metric_params={'V': V})
    else:
        knn = KNeighborsClassifier(n_neighbors=knn_k, metric=distance, weights = weight_function)

    #Train the model using the training sets
    knn.fit(X_train, y_train)

    #Predict the response for test dataset
    y_pred = knn.predict(X_test)

    cm_test = confusion_matrix(y_test, y_pred)

    # Model Accuracy, how often is the classifier correct?
    print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [0]:
def select_bests_params(results):
    df_results = pd.DataFrame(results)
    return df_results.sort_values([3],  ascending=[0])

In [14]:
results = []
for weight_function in weights:
    for knn_k in range(1,5,2):
        for distance in distances:
            print("Função: {} - distancia: {} - K: {}".format(weight_function,distance, 5))
            results.append(cross_validation(knn_k, distance, weight_function))

ranking = select_bests_params(results)
print(ranking)

Função: harmonic - distancia: canberra - K: 5
Treino: 10486, fold_treino: 5856, fold_test: 1489
1489
50 0.0335795836131632
100 0.0671591672263264
150 0.10073875083948959
200 0.1343183344526528
250 0.16789791806581597
300 0.20147750167897918
350 0.23505708529214236
400 0.2686366689053056
450 0.3022162525184688
500 0.33579583613163194
550 0.36937541974479515
600 0.40295500335795836
650 0.4365345869711216
700 0.47011417058428473
750 0.503693754197448
800 0.5372733378106112
850 0.5708529214237743
900 0.6044325050369376
950 0.6380120886501007
1000 0.6715916722632639
1050 0.7051712558764271
1100 0.7387508394895903
1150 0.7723304231027536
1200 0.8059100067159167
1250 0.8394895903290799
1300 0.8730691739422431
1350 0.9066487575554063
1400 0.9402283411685695
1450 0.9738079247817327
Accuracy: 0.9597044996642041


NameError: ignored

In [0]:
best_knn_k, best_distance, best_weight_function, best_avg_acc = list(ranking.head(1).itertuples(index=False, name=None))[0]

In [0]:
best_knn_k, best_distance, best_weight_function

In [0]:
test(best_knn_k, best_distance, best_weight_function)

In [0]:
type(y_test)

#acc = metrics.accuracy_score(fold_y_test, fold_y_pred)